In [ ]:
%load_ext tensorboard

In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
import tensorflow as tf

In [ ]:
in_dim = 256
# out_dim = 256
out_dim = 1

In [ ]:
tf.random.set_seed(0)
tf_norm_init = tf.keras.initializers.get("normal")  # "random_normal"

def tf_init(var):
    var.assign(tf_norm_init(var.shape))

# Compile it ('cause I dunno what else to do it with.)
tf_module = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(in_dim,)),
    tf.keras.layers.Dense(out_dim),
])

tf_dense = tf_module.layers[0]
tf_init(tf_dense.kernel)
tf_init(tf_dense.bias)

In [ ]:
torch.random.manual_seed(0)

def torch_init(p):
    # Match w/ default for normal.
    p.data.normal_(mean=tf_norm_init.mean, std=tf_norm_init.stddev)

torch_dense = torch.nn.Linear(in_features=in_dim, out_features=out_dim)
torch_init(torch_dense.weight)
torch_init(torch_dense.bias)

In [ ]:
log_dir = "/tmp/log"
max_bins = 256
!rm -rf {log_dir}
writer = SummaryWriter(log_dir)
for step in [0, 1]:
    writer.add_histogram("tf/weight", tf_dense.kernel.numpy(), step, max_bins=max_bins)
    writer.add_histogram("tf/bias", tf_dense.bias.numpy(), step, max_bins=max_bins)
    writer.add_histogram("torch/weight", torch_dense.weight.detach().numpy(), step, max_bins=max_bins)
    writer.add_histogram("torch/bias", torch_dense.bias.detach().numpy(), step, max_bins=max_bins)
writer.flush()

In [ ]:
# Kinda weird to see the scalar values listed as a block... but meh.
%tensorboard --logdir {log_dir}